In [1]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from sklearn.metrics import mean_squared_error,median_absolute_error,r2_score,mean_absolute_error, accuracy_score, roc_curve, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

In [2]:
df = pd.read_csv('ebola3.csv', index_col=0)

In [3]:
df.head(2)

,X,District,Poverty_Headcount,Poverty_Gap,Poverty_Severity,Gini_Coefficient,Proportion_Rural,Net_Primarily_Enrollment,Agriculture_Main_Livelihood,Poplation,Area,Population_Density,Borders_Water,Borders_Guinea,Borders_Liberia,Borders_Bo,Borders_Bombali,Borders_Bonthe,Borders_FreeTown,Borders_Kailahun,Borders_Kambia,Borders_Kenema,Borders_Koinadugu,Borders_Kono,Borders_Moyamba,Borders_PortLoko,Borders_Pujehun,Borders_Tonkolili,Borders_Western_Rural,Computer_Penetration,Internet_Penetration,English_Literacy,Native_Language_Literacy,Knows_Cause_of_Malaria,Recently_Ill,Recently_in_Hospital,Purchases_Medicine,Has_Disability,Physically_Weak,Percentage_Vaccinated,Christian,Muslim,Other.Religion,Age_of_Marriage,Attended_School,Employed,Paid_in_Barter,Commutes_to_Work,Migrated_to_Area,Primary_Completion,Junior_Secondary_Completion,Secondary_Completion,Tertiary_Completion,Number_of_Children,Birth_Weight,Contraception_Usage,AIDs_Awareness_Rate,AIDs_Protection_Rate,Condom_Awareness_Rate,Condom_Usage,Average_Commute_Measure,Father_Primary,Father_.Junior_Secondary,Father_Secondary,Father_Tertiary,Mother_Primary,Mother_Junior_Secondary,Mother_Secondary,Mother_Tertiary,Malaria_Prevalence,Cholera_Prevalence,Diarrhea_Prevalence,Dysentry_Prevalence,Guinea_Worm_Prevalence,Infection_Prevalence,River_Blindness_Prevalence,STI_Prevalence,TB_Prevalence,Typhoid_Prevalence,Other_Illness_Prevalence,Count_NGO,Count_NGO_Case_Managment,Count_NGO_Child_Protection,Count_NGO_Communication,Count_NGO_Coordination,Count_NGO_Cross_Border,Count_NGO_Family_Liason,Count_NGO_Food,Count_NGO_Logistics,Count_NGO_Other,Count_NGO_Psychosocial,Count_NGO_Burials,Count_NGO_Social_Mobilization,Count_NGO_Surveillance,Count_NGO_WASH,Epi.week,patient_database_confirmed,situation_report_confirmed,Year,Week,Day,Month,Datetime,PDC_lag1,PDC_lag2,PDC_lag3,PDC_lag4,PDC_lag5,SRC_lag1,SRC_lag2,SRC_lag3,SRC_lag4,SRC_lag5,District_Bo,District_Bombali,District_Bonthe,District_Free.Town,District_Kailahun,District_Kambia,District_Kenema,District_Koinadugu,District_Kono,District_Moyamba,District_Port.Loko,District_Pujehun,District_Tonkolili,District_Western.Rural,case_occured_PDC,case_occured_SRC,case_occured_PDC_lag1,case_occured_PDC_lag2,case_occured_PDC_lag3,case_occured_PDC_lag4,case_occured_PDC_lag5,case_occured_SRC_lag1,case_occured_SRC_lag2,case_occured_SRC_lag3,case_occured_SRC_lag4,case_occured_SRC_lag5,case_ind_Kailahun_PDC_lag1,case_ind_Kailahun_PDC_lag2,case_ind_Kailahun_PDC_lag3,case_ind_Kailahun_PDC_lag4,case_ind_Kailahun_PDC_lag5,case_num_Kailahun_PDC_lag1,case_num_Kailahun_PDC_lag2,case_num_Kailahun_PDC_lag3,case_num_Kailahun_PDC_lag4,case_num_Kailahun_PDC_lag5,case_ind_Kailahun_SRC_lag1,case_ind_Kailahun_SRC_lag2,case_ind_Kailahun_SRC_lag3,case_ind_Kailahun_SRC_lag4,case_ind_Kailahun_SRC_lag5,case_num_Kailahun_SRC_lag1,case_num_Kailahun_SRC_lag2,case_num_Kailahun_SRC_lag3,case_num_Kailahun_SRC_lag4,case_num_Kailahun_SRC_lag5,case_ind_Kono_PDC_lag1,case_ind_Kono_PDC_lag2,case_ind_Kono_PDC_lag3,case_ind_Kono_PDC_lag4,case_ind_Kono_PDC_lag5,case_num_Kono_PDC_lag1,case_num_Kono_PDC_lag2,case_num_Kono_PDC_lag3,case_num_Kono_PDC_lag4,case_num_Kono_PDC_lag5,case_ind_Kono_SRC_lag1,case_ind_Kono_SRC_lag2,case_ind_Kono_SRC_lag3,case_ind_Kono_SRC_lag4,case_ind_Kono_SRC_lag5,case_num_Kono_SRC_lag1,case_num_Kono_SRC_lag2,case_num_Kono_SRC_lag3,case_num_Kono_SRC_lag4,case_num_Kono_SRC_lag5,case_ind_Bombali_PDC_lag1,case_ind_Bombali_PDC_lag2,case_ind_Bombali_PDC_lag3,case_ind_Bombali_PDC_lag4,case_ind_Bombali_PDC_lag5,case_num_Bombali_PDC_lag1,case_num_Bombali_PDC_lag2,case_num_Bombali_PDC_lag3,case_num_Bombali_PDC_lag4,case_num_Bombali_PDC_lag5,case_ind_Bombali_SRC_lag1,case_ind_Bombali_SRC_lag2,case_ind_Bombali_SRC_lag3,case_ind_Bombali_SRC_lag4,case_ind_Bombali_SRC_lag5,case_num_Bombali_SRC_lag1,case_num_Bombali_SRC_lag2,case_num_Bombali_SRC_lag3,case_num_Bombali_SRC_lag4,case_num_Bombali_SRC_lag5,case_ind_Kambia_PDC_lag1,case_ind_Kambia_PDC_lag2,case_ind_Kambia_PDC_lag3,case_ind_Kambia_PD

# Further data cleaning

In [4]:
# Filling all NaN values with 0
df.fillna(0, inplace = True)

In [5]:
# Turning the date to datetime type
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [6]:
# Creating new value for any outbreak
df['any_cases'] = df.situation_report_confirmed.mask(df.situation_report_confirmed > 0, 1)

In [7]:
df.head(2)

,X,District,Poverty_Headcount,Poverty_Gap,Poverty_Severity,Gini_Coefficient,Proportion_Rural,Net_Primarily_Enrollment,Agriculture_Main_Livelihood,Poplation,Area,Population_Density,Borders_Water,Borders_Guinea,Borders_Liberia,Borders_Bo,Borders_Bombali,Borders_Bonthe,Borders_FreeTown,Borders_Kailahun,Borders_Kambia,Borders_Kenema,Borders_Koinadugu,Borders_Kono,Borders_Moyamba,Borders_PortLoko,Borders_Pujehun,Borders_Tonkolili,Borders_Western_Rural,Computer_Penetration,Internet_Penetration,English_Literacy,Native_Language_Literacy,Knows_Cause_of_Malaria,Recently_Ill,Recently_in_Hospital,Purchases_Medicine,Has_Disability,Physically_Weak,Percentage_Vaccinated,Christian,Muslim,Other.Religion,Age_of_Marriage,Attended_School,Employed,Paid_in_Barter,Commutes_to_Work,Migrated_to_Area,Primary_Completion,Junior_Secondary_Completion,Secondary_Completion,Tertiary_Completion,Number_of_Children,Birth_Weight,Contraception_Usage,AIDs_Awareness_Rate,AIDs_Protection_Rate,Condom_Awareness_Rate,Condom_Usage,Average_Commute_Measure,Father_Primary,Father_.Junior_Secondary,Father_Secondary,Father_Tertiary,Mother_Primary,Mother_Junior_Secondary,Mother_Secondary,Mother_Tertiary,Malaria_Prevalence,Cholera_Prevalence,Diarrhea_Prevalence,Dysentry_Prevalence,Guinea_Worm_Prevalence,Infection_Prevalence,River_Blindness_Prevalence,STI_Prevalence,TB_Prevalence,Typhoid_Prevalence,Other_Illness_Prevalence,Count_NGO,Count_NGO_Case_Managment,Count_NGO_Child_Protection,Count_NGO_Communication,Count_NGO_Coordination,Count_NGO_Cross_Border,Count_NGO_Family_Liason,Count_NGO_Food,Count_NGO_Logistics,Count_NGO_Other,Count_NGO_Psychosocial,Count_NGO_Burials,Count_NGO_Social_Mobilization,Count_NGO_Surveillance,Count_NGO_WASH,Epi.week,patient_database_confirmed,situation_report_confirmed,Year,Week,Day,Month,Datetime,PDC_lag1,PDC_lag2,PDC_lag3,PDC_lag4,PDC_lag5,SRC_lag1,SRC_lag2,SRC_lag3,SRC_lag4,SRC_lag5,District_Bo,District_Bombali,District_Bonthe,District_Free.Town,District_Kailahun,District_Kambia,District_Kenema,District_Koinadugu,District_Kono,District_Moyamba,District_Port.Loko,District_Pujehun,District_Tonkolili,District_Western.Rural,case_occured_PDC,case_occured_SRC,case_occured_PDC_lag1,case_occured_PDC_lag2,case_occured_PDC_lag3,case_occured_PDC_lag4,case_occured_PDC_lag5,case_occured_SRC_lag1,case_occured_SRC_lag2,case_occured_SRC_lag3,case_occured_SRC_lag4,case_occured_SRC_lag5,case_ind_Kailahun_PDC_lag1,case_ind_Kailahun_PDC_lag2,case_ind_Kailahun_PDC_lag3,case_ind_Kailahun_PDC_lag4,case_ind_Kailahun_PDC_lag5,case_num_Kailahun_PDC_lag1,case_num_Kailahun_PDC_lag2,case_num_Kailahun_PDC_lag3,case_num_Kailahun_PDC_lag4,case_num_Kailahun_PDC_lag5,case_ind_Kailahun_SRC_lag1,case_ind_Kailahun_SRC_lag2,case_ind_Kailahun_SRC_lag3,case_ind_Kailahun_SRC_lag4,case_ind_Kailahun_SRC_lag5,case_num_Kailahun_SRC_lag1,case_num_Kailahun_SRC_lag2,case_num_Kailahun_SRC_lag3,case_num_Kailahun_SRC_lag4,case_num_Kailahun_SRC_lag5,case_ind_Kono_PDC_lag1,case_ind_Kono_PDC_lag2,case_ind_Kono_PDC_lag3,case_ind_Kono_PDC_lag4,case_ind_Kono_PDC_lag5,case_num_Kono_PDC_lag1,case_num_Kono_PDC_lag2,case_num_Kono_PDC_lag3,case_num_Kono_PDC_lag4,case_num_Kono_PDC_lag5,case_ind_Kono_SRC_lag1,case_ind_Kono_SRC_lag2,case_ind_Kono_SRC_lag3,case_ind_Kono_SRC_lag4,case_ind_Kono_SRC_lag5,case_num_Kono_SRC_lag1,case_num_Kono_SRC_lag2,case_num_Kono_SRC_lag3,case_num_Kono_SRC_lag4,case_num_Kono_SRC_lag5,case_ind_Bombali_PDC_lag1,case_ind_Bombali_PDC_lag2,case_ind_Bombali_PDC_lag3,case_ind_Bombali_PDC_lag4,case_ind_Bombali_PDC_lag5,case_num_Bombali_PDC_lag1,case_num_Bombali_PDC_lag2,case_num_Bombali_PDC_lag3,case_num_Bombali_PDC_lag4,case_num_Bombali_PDC_lag5,case_ind_Bombali_SRC_lag1,case_ind_Bombali_SRC_lag2,case_ind_Bombali_SRC_lag3,case_ind_Bombali_SRC_lag4,case_ind_Bombali_SRC_lag5,case_num_Bombali_SRC_lag1,case_num_Bombali_SRC_lag2,case_num_Bombali_SRC_lag3,case_num_Bombali_SRC_lag4,case_num_Bombali_SRC_lag5,case_ind_Kambia_PDC_lag1,case_ind_Kambia_PDC_lag2,case_ind_Kambia_PDC_lag3,case_ind_Kambia_PD

# Ensemble models

In [8]:
selected_weeks = ['2014-09-06', '2014-10-27','2014-12-29']

df_holdout = df[df.Datetime.isin(selected_weeks)]
df2 = df[~df.Datetime.isin(selected_weeks)]
   
print(df.shape)
print(df_holdout.shape)
print(df2.shape)

(1722, 656)
(41, 656)
(1681, 656)


In [9]:
# Preparing the dataset by removing irrelevant variables
df2.drop(columns = ['X', 'District', 'patient_database_confirmed', 'Year', 'Week', 'Day'], inplace = True)


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
df2.drop(columns = ['Epi.week'], inplace = True)


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [11]:
df2.drop(columns = ['case_occured_PDC', 'case_occured_SRC'], inplace = True)

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1681 entries, 1 to 1722
Columns: 647 entries, Poverty_Headcount to any_cases
dtypes: datetime64[ns](1), float64(598), int64(48)
memory usage: 8.3 MB


In [13]:
df2.head(2)

,Poverty_Headcount,Poverty_Gap,Poverty_Severity,Gini_Coefficient,Proportion_Rural,Net_Primarily_Enrollment,Agriculture_Main_Livelihood,Poplation,Area,Population_Density,Borders_Water,Borders_Guinea,Borders_Liberia,Borders_Bo,Borders_Bombali,Borders_Bonthe,Borders_FreeTown,Borders_Kailahun,Borders_Kambia,Borders_Kenema,Borders_Koinadugu,Borders_Kono,Borders_Moyamba,Borders_PortLoko,Borders_Pujehun,Borders_Tonkolili,Borders_Western_Rural,Computer_Penetration,Internet_Penetration,English_Literacy,Native_Language_Literacy,Knows_Cause_of_Malaria,Recently_Ill,Recently_in_Hospital,Purchases_Medicine,Has_Disability,Physically_Weak,Percentage_Vaccinated,Christian,Muslim,Other.Religion,Age_of_Marriage,Attended_School,Employed,Paid_in_Barter,Commutes_to_Work,Migrated_to_Area,Primary_Completion,Junior_Secondary_Completion,Secondary_Completion,Tertiary_Completion,Number_of_Children,Birth_Weight,Contraception_Usage,AIDs_Awareness_Rate,AIDs_Protection_Rate,Condom_Awareness_Rate,Condom_Usage,Average_Commute_Measure,Father_Primary,Father_.Junior_Secondary,Father_Secondary,Father_Tertiary,Mother_Primary,Mother_Junior_Secondary,Mother_Secondary,Mother_Tertiary,Malaria_Prevalence,Cholera_Prevalence,Diarrhea_Prevalence,Dysentry_Prevalence,Guinea_Worm_Prevalence,Infection_Prevalence,River_Blindness_Prevalence,STI_Prevalence,TB_Prevalence,Typhoid_Prevalence,Other_Illness_Prevalence,Count_NGO,Count_NGO_Case_Managment,Count_NGO_Child_Protection,Count_NGO_Communication,Count_NGO_Coordination,Count_NGO_Cross_Border,Count_NGO_Family_Liason,Count_NGO_Food,Count_NGO_Logistics,Count_NGO_Other,Count_NGO_Psychosocial,Count_NGO_Burials,Count_NGO_Social_Mobilization,Count_NGO_Surveillance,Count_NGO_WASH,situation_report_confirmed,Month,Datetime,PDC_lag1,PDC_lag2,PDC_lag3,PDC_lag4,PDC_lag5,SRC_lag1,SRC_lag2,SRC_lag3,SRC_lag4,SRC_lag5,District_Bo,District_Bombali,District_Bonthe,District_Free.Town,District_Kailahun,District_Kambia,District_Kenema,District_Koinadugu,District_Kono,District_Moyamba,District_Port.Loko,District_Pujehun,District_Tonkolili,District_Western.Rural,case_occured_PDC_lag1,case_occured_PDC_lag2,case_occured_PDC_lag3,case_occured_PDC_lag4,case_occured_PDC_lag5,case_occured_SRC_lag1,case_occured_SRC_lag2,case_occured_SRC_lag3,case_occured_SRC_lag4,case_occured_SRC_lag5,case_ind_Kailahun_PDC_lag1,case_ind_Kailahun_PDC_lag2,case_ind_Kailahun_PDC_lag3,case_ind_Kailahun_PDC_lag4,case_ind_Kailahun_PDC_lag5,case_num_Kailahun_PDC_lag1,case_num_Kailahun_PDC_lag2,case_num_Kailahun_PDC_lag3,case_num_Kailahun_PDC_lag4,case_num_Kailahun_PDC_lag5,case_ind_Kailahun_SRC_lag1,case_ind_Kailahun_SRC_lag2,case_ind_Kailahun_SRC_lag3,case_ind_Kailahun_SRC_lag4,case_ind_Kailahun_SRC_lag5,case_num_Kailahun_SRC_lag1,case_num_Kailahun_SRC_lag2,case_num_Kailahun_SRC_lag3,case_num_Kailahun_SRC_lag4,case_num_Kailahun_SRC_lag5,case_ind_Kono_PDC_lag1,case_ind_Kono_PDC_lag2,case_ind_Kono_PDC_lag3,case_ind_Kono_PDC_lag4,case_ind_Kono_PDC_lag5,case_num_Kono_PDC_lag1,case_num_Kono_PDC_lag2,case_num_Kono_PDC_lag3,case_num_Kono_PDC_lag4,case_num_Kono_PDC_lag5,case_ind_Kono_SRC_lag1,case_ind_Kono_SRC_lag2,case_ind_Kono_SRC_lag3,case_ind_Kono_SRC_lag4,case_ind_Kono_SRC_lag5,case_num_Kono_SRC_lag1,case_num_Kono_SRC_lag2,case_num_Kono_SRC_lag3,case_num_Kono_SRC_lag4,case_num_Kono_SRC_lag5,case_ind_Bombali_PDC_lag1,case_ind_Bombali_PDC_lag2,case_ind_Bombali_PDC_lag3,case_ind_Bombali_PDC_lag4,case_ind_Bombali_PDC_lag5,case_num_Bombali_PDC_lag1,case_num_Bombali_PDC_lag2,case_num_Bombali_PDC_lag3,case_num_Bombali_PDC_lag4,case_num_Bombali_PDC_lag5,case_ind_Bombali_SRC_lag1,case_ind_Bombali_SRC_lag2,case_ind_Bombali_SRC_lag3,case_ind_Bombali_SRC_lag4,case_ind_Bombali_SRC_lag5,case_num_Bombali_SRC_lag1,case_num_Bombali_SRC_lag2,case_num_Bombali_SRC_lag3,case_num_Bombali_SRC_lag4,case_num_Bombali_SRC_lag5,case_ind_Kambia_PDC_lag1,case_ind_Kambia_PDC_lag2,case_ind_Kambia_PDC_lag3,case_ind_Kambia_PDC_lag4,case_ind_Kambia_PDC_lag5,case_num_Kambia_PDC_lag1,case_num_Kambia_PDC_lag2,case_num_Kamb

In [14]:
df2.situation_report_confirmed.sum()

6471

In [15]:
X = df2.drop(columns = ['any_cases', 'Datetime', 'situation_report_confirmed'])
y = df2.situation_report_confirmed

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Random Forest

In [17]:
random_forest = RandomForestRegressor(n_estimators = 100)
random_forest.fit(X_train, y_train)
mean_squared_error(y_test, random_forest.predict(X_test))

39.19869806125646

In [18]:
gsc = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid={
        'max_depth': range(3,7),
        'n_estimators': (10, 50, 100, 1000),
    }, cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
grid_result = gsc.fit(X, y)
best_params = grid_result.best_params_
    
rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],                               random_state=False, verbose=False)# Perform K-Fold CV
scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')

np.mean(scores)

-2.683978744676771

In [19]:
rfr.fit(X_train, y_train)
preds_rfr = rfr.predict(X_test)
mean_squared_error(y_test, preds_rfr)

42.55984759966684

### XGBoost

In [17]:
xgb_model = xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 



In [18]:
xgb_model.fit(X_train, y_train)
mean_squared_error(y_test, xgb_model.predict(X_test))

C:\Users\User\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:17:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


36.440627361764385

In [ ]:
'''
#for tuning parameters
parameters_for_testing = {
    'colsample_bytree':[0.4,0.6,0.8],
    'gamma':[0,0.03,0.1,0.3],
    'min_child_weight':[1.5,6,10],
    'learning_rate':[0.1,0.07],
    'max_depth':[3,5],
    'n_estimators':[10000],
    'reg_alpha':[1e-5, 1e-2,  0.75],
    'reg_lambda':[1e-5, 1e-2, 0.45],
    'subsample':[0.6,0.95]  
}

                    
xgb_model = xgb.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5,
     min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)

gsearch1 = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_absolute_error')
gsearch1.fit(X_train,y_train)
print (gsearch1.grid_scores_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)
'''


Fitting 3 folds for each of 2592 candidates, totalling 7776 fits


In [38]:
#for tuning parameters
parameters_for_testing = {
    'colsample_bytree':[0.4,0.8],
    'gamma':[0, 0.3],
    'min_child_weight':[1.5, 10],
    'learning_rate':[0.1],
    'max_depth':[3],
    'n_estimators':[1000],
    'reg_alpha':[0.75],
    'reg_lambda':[1e-5, 0.45],
    'subsample':[0.6,0.95]  
}

                    
xgb_model = xgb.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5,
     min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)

gsearch1 = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_absolute_error')
gsearch1.fit(X_train,y_train)
print (gsearch1.grid_scores_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   49.9s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:  1.0min
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:  2.9min
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:  3.8min
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  4.6min
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  6.0min
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:  8.1min
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:  9.7min
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed: 13.1min
[Parallel(n_jobs=6)]: Done  96 out of  96 | elapsed: 17.2min finished
C:\Users\User\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:22:48] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[mean: -1.90081, std: 0.35091, params: {'colsample_bytree': 0.4, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1.5, 'n_estimators': 1000, 'reg_alpha': 0.75, 'reg_lambda': 1e-05, 'subsample': 0.6}, mean: -1.80370, std: 0.32333, params: {'colsample_bytree': 0.4, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1.5, 'n_estimators': 1000, 'reg_alpha': 0.75, 'reg_lambda': 1e-05, 'subsample': 0.95}, mean: -1.83874, std: 0.35575, params: {'colsample_bytree': 0.4, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1.5, 'n_estimators': 1000, 'reg_alpha': 0.75, 'reg_lambda': 0.45, 'subsample': 0.6}, mean: -1.78088, std: 0.33287, params: {'colsample_bytree': 0.4, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1.5, 'n_estimators': 1000, 'reg_alpha': 0

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [39]:
best_xgb_model = xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=.30,                 
                 learning_rate=0.1,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=1000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.95,
                 seed=42)
best_xgb_model.fit(X_train, y_train)
best_xgb_preds = best_xgb_model.predict(X_test)
mean_squared_error(y_test, best_xgb_preds)

C:\Users\User\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:23:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


44.14884219678926

### Saving the results

In [20]:
X_holdout = df_holdout.drop(columns = ['any_cases', 'Datetime', 'situation_report_confirmed', 'X', 'District', 'Epi.week', 'patient_database_confirmed', 'Year', 'Week', 'Day', 'case_occured_PDC', 'case_occured_SRC'])
y_holdout = df_holdout.situation_report_confirmed


In [22]:
df_holdout['prediction'] = random_forest.predict(X_holdout)
results = df_holdout[['District', 'Datetime', 'situation_report_confirmed', 'prediction']]
results.head(20)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,District,Datetime,situation_report_confirmed,prediction
Column1,,,,
323,Bo,2014-09-06,0,0.00
324,Bombali,2014-09-06,0,0.00
325,Bonthe,2014-09-06,0,0.00
326,Free Town,2014-09-06,0,0.00
327,Kailahun,2014-09-06,0,0.20
328,Kambia,2014-09-06,0,0.00
329,Kenema,2014-09-06,0,0.00
330,Koinadugu,2014-09-06,0,0.00
331,Kono,2014-09-06,0,0.00


In [28]:
results.to_csv('regression_results.csv')